# Nautilus Persistent Volume Creation

This notebook will help you create a persistent volume on the Nautilus cluster

## Step 0: Prerequisites
* You have been added to a Nautilus namespace
* You have cloned down the Big Data repository

## Step 1: Load the Template YAML File

We will be using a template of a PVC creation YAML to make this process easy:

```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: {{ name }}
spec:
  storageClassName: rook-cephfs-central
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: 50Gi
```

We are creating a persistent volume in the rook CephFS central storage pool with 50 GB of storage.

Let's load our template using a template engine called `jinja2`:

In [6]:
from jinja2 import Template

# read in the template
with open('../yaml/pvc_template.yml') as file_:
    template = Template(file_.read())

## Step 2: Render and Save the YAML File

Now, we can replace the name of the PVC we are going to create, and save the YAML File. 

Be sure to use something like your initials in the name of the PVC. Every namespace can only have a single PVC with a given name.

In [7]:
# replace None 
PVC_NAME = None

pvc_spec = template.render(name=PVC_NAME)

Let's view our rendered YAML:

In [8]:
print(pvc_spec)

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: gp-engine-unoselab01-pvc
spec:
  storageClassName: rook-cephfs-central
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: 50Gi


Finally, let's save the YAML to disk:

In [10]:
with open("./pvc_create.yml", "w") as file:
    file.write(pvc_spec)

## Step 3: Create the PVC

Now, we need to invoke KubeCTL with our saved YAML file to create the PVC

In [11]:
! kubectl create -f ./pvc_create.yml

persistentvolumeclaim/gp-engine-unoselab01-pvc created


## Step 4: Verify PVC Creation

The output of Step 3 should have been something such as:
```
persistentvolumeclaim/YOURNAME created
```

Let's view all of the PVC in our namespace and ensure we find ours:

In [13]:
! kubectl get pvc

NAME                       STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS          AGE
pvc-gp-engine-unoselab01   Bound    pvc-3c27adfd-d9aa-45ff-af1a-c0232610bf5d   50Gi       RWX            rook-cephfs-central   3h43m
